# Fast and efficient preprocessing of scRNA-seq with kallisto | bustools | kb-python


This notebook provides a complete workflow to quantify single-cell RNA-seq data using using **kb-python** (kallisto|bustools).


[Md. Jubayer Hossain](https://mdjubayerhossain.com/)

Founder & CEO, [DeepBio Ltd.](https://deepbioltd.com/)

## Step 1: Setup

In [ ]:
# check working directory 
!pwd 

In [ ]:
# Set up main working directory
import os
working_dir = "/Users/jubayer/Teaching/sc-kallisto/"
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

In [ ]:
# Define folder structure
folders = [
    "reference",
    "raw_data",
    "processed_data",
    "results",
    "results/figures",
    "results/tables"
]

# Create folders
for folder in folders:
    path = os.path.join(working_dir, folder)
    os.makedirs(path, exist_ok=True)

In [ ]:
# Set Save paths for easy use later
reference_dir = os.path.join(working_dir, "reference")
raw_data_dir = os.path.join(working_dir, "raw_data")
processed_data_dir = os.path.join(working_dir, "processed_data")
results_dir = os.path.join(working_dir, "results")
figures_dir = os.path.join(working_dir, "results/figures")
tables_dir = os.path.join(working_dir, "results/tables")

## Step 2: Build Reference Index

We'll use kb-python's built-in reference download for mouse. This will download the mouse transcriptome and create an index.

Before we can quantify gene expression from raw FASTQ files, we need to create a reference index.
This index is essential because tools like kb-python (Kallisto | Bustools) must know:

- Which transcripts exist in the organism?
- Where each transcript starts and ends?
- Which transcripts map to which genes?

This allows the reads to be pseudoaligned quickly and accurately.

In [ ]:
# Check directory path
!pwd

In [ ]:
# List directories
!ls

In [ ]:
# Check results
!ls results

In [ ]:
%%time 
# Download and build mouse reference (this may take 10-15 minutes)
# For human: -d human
# For Mouse: -d mouse
!kb ref -d mouse -i reference/index.idx -g reference/t2g.txt

This command downloads and builds the human transcriptome reference needed for `pseudoalignment` and count matrix generation in single-cell RNA-seq.

- `kb ref` prepares everything needed for kb count, including:
    - downloading the transcriptome
    - generating the kallisto index
    - building the transcript-to-gene (t2g) mapping file

- `-d mouse`
    - Tells `kb-python` to automatically download a predefined HUMAN reference dataset.
    - No need to manually supply FASTA or GTF files.

- `-i reference/index.idx`
    - Specifies where to save the kallisto index, which is used for fast pseudoalignment.
- `-g reference/t2g.txt`
    - Creates a transcript-to-gene mapping file.
    - Bustools uses this file to convert transcript counts into gene counts.

### Alternative: Manual Reference Building

If you prefer to build the reference manually from specific genome and GTF files:

In [ ]:
# OPTIONAL: Manual reference building (uncomment to use)
# Download genome and GTF from Ensembl
# !wget ftp://ftp.ensembl.org/pub/release-109/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz
# !wget ftp://ftp.ensembl.org/pub/release-109/gtf/mus_musculus/Mus_musculus.GRCm39.109.gtf.gz

# Build reference
# !kb ref \
#   -i reference/index.idx \
#   -g reference/t2g.txt \
#   -f1 reference/transcriptome.fa \
#   Mus_musculus.GRCm39.dna.primary_assembly.fa.gz \
#   Mus_musculus.GRCm39.109.gtf.gz

## Step 3: Downloading Data 

In [ ]:
# Get sample information
import os
import json
import subprocess

# Sample IDs from the SRA project
# Data URL: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE126783
samples = [
    "SRR10202213", 
    "SRR10202214"
]

# Total samples
print(f"Total samples: {len(samples)}")

print("Samples to download:")
for sample in samples:
    print(f"  - {sample}")

In [ ]:
%%time 
# Download FASTQ files using ffq
# Note: This will download large files (20-40 GB total)
# Make sure you have sufficient disk space and time
for sample in samples:
    print(f"\nDownloading {sample}...")

    # Get FTP URLs using ffq
    result = subprocess.run(
        ["ffq", "--ftp", sample],
        capture_output=True,
        text=True
    )

    # Parse the JSON output
    data = json.loads(result.stdout)

    # Download FASTQ files
    for entry in data:
        url = entry['url']
        filename = os.path.basename(url)
        output_path = f"raw_data/{filename}"

        if not os.path.exists(output_path):
            print(f"Downloading: {filename}")
            !wget -q --show-progress -O {output_path} {url}
        else:
            print(f"File already exists: {filename}")

### Alternative: Use SRA Toolkit (if ffq has issues)

In [ ]:
# ALTERNATIVE: Download using SRA toolkit
# !pip install sra-tools -q

# for sample in samples:
#     print(f"Downloading {sample}...")
#     !fastq-dump --split-files --gzip --outdir raw_data {sample}

## Step 4: Quantify with kb-python (Pseudoalignment and UMI counting)

Now we'll use kb count to process the FASTQ files. Since this is 10x Chromium data, we'll use the `10xv2/v3` technology specification.

In [ ]:
%%time 
# Process each sample
# This step runs `kb` to pseudoalign the reads, and then generate the cells x gene matrix in h5ad format.
for sample in samples:
    print(f"\n{'='*60}")
    print(f"Processing sample: {sample}")
    print(f"{'='*60}\n")

    # Define input and output paths
    r1 = f"raw_data/{sample}_1.fastq.gz"
    r2 = f"raw_data/{sample}_2.fastq.gz"
    output_dir = f"output/{sample}"

    # Run kb count
    # Note: Adjust -x parameter if needed (try 10xv3 if 10xv2 doesn't work well)
    !kb count \
        -i reference/index.idx \
        -g reference/t2g.txt \
        -x 10xv3 \
        -o {output_dir} \
        --h5ad \
        -t 4 \
        {r1} {r2}

    print(f"\nCompleted processing: {sample}")
    print(f"Output directory: {output_dir}")

**Understanding kb count parameters:**

- `-i`: Index file created in Step 2
- `-g`: Transcript-to-gene mapping file
- `-x 10xv3`: Technology specification (10x Chromium v3)
- `-o`: Output directory
- `--h5ad`: Generate AnnData h5ad file for easy loading in Python
- `-t 4`: Use 4 threads (adjust based on available resources)
- Last arguments: Read 1 and Read 2 FASTQ files